## Polynomial Yield Curve Fitting

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from numpy.polynomial.polynomial import Polynomial
import os
from dotenv import load_dotenv
from fredapi import Fred
load_dotenv()

fred = Fred(api_key=os.getenv("FRED_API_KEY"))

In [9]:
def get_yields(start=None):
    df = pd.DataFrame()

    yields = {
        'DGS1MO': 'US1M',
        'DGS3MO': 'US3M',
        'DGS6MO': 'US6M',
        'DGS1': 'US1Y',
        'DGS2': 'US2Y',
        'DGS5': 'US5Y',
        'DGS7': 'US7Y',
        'DGS10': 'US10Y',
        'DGS20': 'US20Y',
        'DGS30': 'US30Y'
    }

    for ticker, tenor in yields.items():
        series = fred.get_series(ticker)
        series_df = pd.DataFrame(series, columns=[tenor])
        series_df.reset_index(inplace=True)
        series_df.rename(columns={'index': 'Date'}, inplace=True)

        if df.empty:
            df = series_df
        else:
            df = pd.merge(df, series_df, on='Date', how='outer')

    df.set_index('Date', inplace=True)
    df.sort_index(ascending=False, inplace=True)

    latest_date = df.index.max()
    one_wk = latest_date - pd.Timedelta(weeks=1)
    one_mth = latest_date - pd.DateOffset(months=1)

    def get_nearest_index(target_date):
        return df.index[df.index.get_indexer([target_date], method='nearest')[0]]

    current = df.loc[latest_date]
    wk_ago = df.loc[get_nearest_index(one_wk)]
    mth_ago = df.loc[get_nearest_index(one_mth)]

    tenor_to_years = {
        "US1M": 1/12,
        "US3M": 0.25,
        "US6M": 0.5,
        "US1Y": 1,
        "US2Y": 2,
        "US5Y": 5,
        "US7Y": 7,
        "US10Y": 10,
        "US20Y": 20,
        "US30Y": 30,
    }

    yield_curve = pd.DataFrame({
        'Maturity': df.columns.tolist() * 3,
        'Yield': pd.concat([current, wk_ago, mth_ago], axis=0).values,
        'Date': ['Current'] * len(df.columns) + ['1 W'] * len(df.columns) + ['1 M'] * len(df.columns)
    })

    yield_curve['Maturity_Years'] = yield_curve['Maturity'].map(tenor_to_years)

    yiled_curve_fig = px.line(yield_curve,
                  x='Maturity_Years',
                  y='Yield',
                  color='Date',
                  markers=True,
                  template='plotly_dark',
                  title="US Yield Curve: Current / 1W Ago / 1M Ago")

    yiled_curve_fig.update_layout(xaxis_title='Maturity (Years)', yaxis_title='Yield (%)')
    
    current_curve = yield_curve[yield_curve['Date'] == 'Current'].dropna()
    x = current_curve['Maturity_Years'].values
    y = current_curve['Yield'].values

    coeffs = Polynomial.fit(x, y, deg=3).convert().coef

    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = np.polyval(coeffs[::-1], x_fit)

    yiled_curve_fig.add_scatter(x=x_fit,
                                y=y_fit,
                                mode='lines',
                                name='Fitted Yield Curve',
                                line=dict(dash='dash', color='cyan'))

    yiled_curve_fig.update_layout(xaxis_title='Maturity (Years)', yaxis_title='Yield (%)')
    yiled_curve_fig.show()

    current_curve = yield_curve[yield_curve['Date'] == 'Current'].dropna()
    x = current_curve['Maturity_Years'].values
    y = current_curve['Yield'].values

    coeffs = Polynomial.fit(x, y, deg=3).convert().coef  # coeffs from lowest to highest power
    fitted_y = np.polyval(coeffs[::-1], x)  # polyval expects highest-degree first

    deviation_df = pd.DataFrame({
        'Maturity': current_curve['Maturity'].values,
        'Maturity_Years': x,
        'Actual_Yield': y,
        'Fitted_Yield': fitted_y,
        'Deviation': y - fitted_y
    }).sort_values(by='Deviation', key=np.abs, ascending=False)

    print("\nTop deviations from fitted curve:")
    print(deviation_df[['Maturity', 'Actual_Yield', 'Fitted_Yield', 'Deviation']])

    return df, deviation_df

historical_yields, current_poly_fit = get_yields()


Top deviations from fitted curve:
  Maturity  Actual_Yield  Fitted_Yield  Deviation
4     US2Y          4.02      4.204834  -0.184834
3     US1Y          4.11      4.241414  -0.131414
1     US3M          4.41      4.280167   0.129833
7    US10Y          4.49      4.380392   0.109608
0     US1M          4.39      4.290160   0.099840
5     US5Y          4.12      4.186181  -0.066181
6     US7Y          4.30      4.236597   0.063403
8    US20Y          4.97      5.026308  -0.056308
2     US6M          4.29      4.266131   0.023869
9    US30Y          4.94      4.927815   0.012185
